In [1]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup

In [60]:
url = 'https://stockanalysis.com/list/biggest-companies/'
response = requests.get(url)
html_content = response.content
soup = BeautifulSoup(html_content, 'html.parser')
table = soup.find_all('table') 

In [61]:
table_data = []
for row in table[0].find_all('tr'):
    row_data = [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])]  # Extract text from each cell
    table_data.append(row_data)

In [65]:
us_companies = (pd.DataFrame(data = table_data[1:], columns = table_data[0])).drop(columns = ['No.', 'Market Cap', '% Change', 'Revenue','Stock Price'])

In [66]:
us_companies

,Symbol,Company Name
0,MSFT,Microsoft Corporation
1,AAPL,Apple Inc
2,NVDA,NVIDIA Corporation
3,AMZN,"Amazon.com, Inc."
4,GOOGL,Alphabet Inc.
...,...,...
495,ILMN,"Illumina, Inc."
496,INVH,Invitation Homes Inc.
497,TECK,Teck Resources Limited
498,CCL,Carnival Corporation & plc


In [28]:
url = 'https://companiesmarketcap.com/european-union/largest-companies-in-the-eu-by-market-cap/'
response = requests.get(url)
html_content = response.content
soup = BeautifulSoup(html_content, 'html.parser')
table = soup.find_all('tbody') 

In [42]:
companies = []
for c in table[0].find_all('div', class_='company-name'):
    companies.append(c.text)

In [47]:
symbols = []
for s in table[0].find_all('div', class_='company-code'):
    symbols.append(s.text)

In [49]:
len(symbols), len(companies)

(100, 100)

In [67]:
eu_companies = pd.DataFrame()
eu_companies['Symbol'] = symbols
eu_companies['Company Name'] = companies

In [ ]:
#extracting final ESG data for US and EU companies

In [138]:
from selenium import webdriver   # driver de selenium
from selenium.webdriver.chrome.options import Options   # opciones del driver
from selenium.webdriver.common.by import By # By es para buscar por tag, clase, id...
from selenium.webdriver.common.keys import Keys  # para usar teclado
from selenium.webdriver.chrome.service import Service
import time

In [213]:
serv = Service('/Users/paloma/Desktop/Ironhack/proyecto_final/chromedriver')

In [214]:
opciones=Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('user-data-dir=cookies')
opciones.add_argument('--disk-cache-dir=/path/to/cache')

In [215]:
driver.quit()

In [216]:
driver = webdriver.Chrome(service = serv, options = opciones)

In [209]:
def list_esg_symbol_scrape(list):
    for s in list:
        driver.get(f'https://finance.yahoo.com/quote/{s}/sustainability')
        time.sleep(5)
        driver.execute_script("window.stop();")
        table = driver.find_element(By.ID, 'Main')
        html = table.get_attribute('outerHTML')
        soup = BeautifulSoup(html, 'html.parser')
        try:
            scores = soup.find_all('div', class_='D(ib) Fz(23px) smartphone_Fz(22px) Fw(600)')
            environment.append(scores[0].text)
            social.append(scores[1].text)
            governance.append(scores[2].text)
            symb.append(s)
        except:
            symb.append(s)
            environment.append('NA')
            social.append('NA')
            governance.append('NA')

In [159]:
symb = []
environment = []
social = []
governance = []

In [ ]:
list_esg_symbol_scrape(eu_companies['Symbol'])

In [197]:
len(symb), len(environment), len(social), len(governance)

(99, 99, 99, 99)

In [199]:
symbols[-1]

'ICLR'

In [200]:
additional_df = pd.DataFrame()
additional_df['Symbol'] = symb
additional_df['Environmental score'] = environment
additional_df['Social score'] = social
additional_df['Governance score'] = governance

In [201]:
additional_df

,Symbol,Environmental score,Social score,Governance score
0,NVO,NA,NA,NA
1,MC.PA,0.8,5.7,5.6
2,ASML,NA,NA,NA
3,RMS.PA,0.9,3.7,4.8
4,OR.PA,4.4,7.2,7.2
...,...,...,...,...
94,FER.MC,7.8,11.4,7.1
95,NZYM.VI,5.3,5.4,5.6
96,DB,NA,NA,NA
97,BAYN.DE,2.8,17.9,6.7


In [202]:
esg_score_eu =pd.merge(eu_companies, additional_df, on='Symbol', how='left')

In [203]:
esg_score_eu

,Symbol,Company Name,Environmental score,Social score,Governance score
0,NVO,Novo Nordisk,NA,NA,NA
1,MC.PA,LVMH,0.8,5.7,5.6
2,ASML,ASML,NA,NA,NA
3,RMS.PA,Hermès,0.9,3.7,4.8
4,OR.PA,L'Oréal,4.4,7.2,7.2
...,...,...,...,...,...
95,FER.MC,Ferrovial,7.8,11.4,7.1
96,NZYM.VI,Novozymes,5.3,5.4,5.6
97,DB,Deutsche Bank,NA,NA,NA
98,BAYN.DE,Bayer,2.8,17.9,6.7


In [211]:
symb = []
environment = []
social = []
governance = []

In [217]:
list_esg_symbol_scrape(us_companies['Symbol'])

In [227]:
additional_df_us = pd.DataFrame()
additional_df_us['Symbol'] = symb
additional_df_us['Environmental score'] = environment
additional_df_us['Social score'] = social
additional_df_us['Governance score'] = governance

In [219]:
esg_score_us =pd.merge(us_companies, additional_df_us, on='Symbol', how='left')

In [223]:
esg_score_eu = pd.read_excel('data/working data/esg_score.xlsx')

In [229]:
esg_score_us.rename(columns ={'Symbol':'Symbols'}, inplace = True)

In [230]:
esg_score = pd.concat([esg_score_eu,esg_score_us], ignore_index = True)

In [231]:
esg_score

,Symbols,Company Name,Environmental score,Social score,Governance score
0,NVO,Novo Nordisk,NaN,NaN,NaN
1,MC.PA,LVMH Moët Hennessy Louis Vuitton SE,0.8,5.7,5.6
2,ASML,ASML Holding N.V.,3.3,2.0,4.3
3,RMS.PA,Hermès International SCA,0.9,3.7,4.8
4,OR.PA,L'Oréal SA,4.4,7.2,7.2
...,...,...,...,...,...
610,ILMN,"Illumina, Inc.",0.1,11.6,6.9
611,INVH,Invitation Homes Inc.,NA,NA,NA
612,TECK,Teck Resources Limited,10.1,6.7,3.6
613,CCL,Carnival Corporation & plc,9.4,10.1,5.0


In [232]:
#esg_score.to_excel('esg_score.xlsx', index = False)